In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained model, scaler pickle , one hot
model = load_model("model.h5")

# load_model the encoder and scaler
with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

with open("one_hot_encoder_geography.pkl", "rb") as file:
    one_hot_encoder_geo = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)


In [3]:
# example input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
}

In [4]:
print(input_data.keys())

dict_keys(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])


In [5]:
data = pd.DataFrame(input_data, index=[0])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
# Assuming you have a list of all possible geography values
all_possible_geographies = ["France", "Germany", "Spain"]

# Fit the encoder with all possible categories
one_hot_encoder_geo.fit(pd.DataFrame({"Geography": all_possible_geographies}))

# Transform the data
data_geo = one_hot_encoder_geo.transform(data[["Geography"]]).toarray()
encoded_geo_data = pd.DataFrame(data_geo, columns=one_hot_encoder_geo.get_feature_names_out())
encoded_geo_data.head()


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
data["Gender"] = label_encoder_gender.transform(data["Gender"])

In [9]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [10]:
data = pd.concat([data, encoded_geo_data], axis=1)
data.drop("Geography", axis=1, inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
scaled_data = scaler.transform(data)
print(scaled_data)

[[-0.54204762  0.91055421  0.10473081 -0.68894811 -0.26230046  0.81966266
   0.64598061  0.97071435 -0.88153859  0.99828718 -0.57559072 -0.57779016]]


In [12]:
pred = model.predict(scaled_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step


In [13]:
pred

array([[0.02668995]], dtype=float32)

In [14]:
pred_probability = pred[0][0]

In [15]:
if pred_probability > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
